<a href="https://www.kaggle.com/code/nazaninmahmoudy/notebook1f8f3c50e0?scriptVersionId=265070027" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🔐 File Integrity Checker

##### A tool in Python to monitor file changes, detect missing or modified files, create backups of altered files, generate CSV reports, and restore files to their original state. This is done by calculating the SHA-256 hash for each file to be monitored and saving these hashes in a JSON file as a baseline dataset. Each time a check is performed, the current SHA-256 hashes are computed and compared to the stored values to detect changes or missing files. In addition, it creates backups of altered files while preserving the directory structure, generates detailed CSV reports with timestamps and hash values, and can restore files to their original state from backups.



## Importing Libraries

In [1]:
import os
import hashlib
import json
import shutil
import csv
from datetime import datetime

In [2]:
def calculate_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            sha256.update(chunk)
    return sha256.hexdigest()


A hash is a unique fixed-length string (like a digital fingerprint) generated from data. If the file changes even a bit, its hash will also change.
The function opens the file in binary mode, reads it chunk by chunk, feeds those chunks into a SHA-256 algorithm, and in the end spits out a unique hash string.

In [3]:
def build_database(directory, db_file="integrity_db.json"):
    db = {}
    for root, _, files in os.walk(directory):
        for file in files:
            path = os.path.join(root, file)
            db[path] = calculate_hash(path)
    with open(db_file, "w") as f:
        json.dump(db, f, indent=4)
    print(f"[+] Database created and saved to {db_file}")


This function foes through all files in the given folder, calculates a hash for each one, and stores the results in a JSON file .  like creating a baseline database of our files so we can check later if anything changes.

In [4]:
def check_integrity(db_file="integrity_db.json", backup_dir=None, report_csv=None):
    if not os.path.exists(db_file):
        print("[-] Database not found. Build database first.")
        return

    with open(db_file, "r") as f:
        db = json.load(f)

    changes = []
    for path, old_hash in db.items():
        status = "No Changes"
        new_hash = old_hash
        if not os.path.exists(path):
            status = "MISSING"
        else:
            new_hash = calculate_hash(path)
            if new_hash != old_hash:
                status = "Changed"
                if backup_dir:
                    rel_path = os.path.relpath(path, start=os.getcwd())
                    backup_path = os.path.join(backup_dir, rel_path)
                    os.makedirs(os.path.dirname(backup_path), exist_ok=True)
                    shutil.copy2(path, backup_path)

        changes.append({
            "file_path": path,
            "status": status,
            "old_hash": old_hash,
            "new_hash": new_hash,
            "checked_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        })
        print(f"[{status}] {path}")

    if report_csv:
        report_csv = report_csv.strip() or None
        if report_csv:
            if os.path.isdir(report_csv):
                report_csv = os.path.join(report_csv, "report.csv")
            os.makedirs(os.path.dirname(report_csv), exist_ok=True)
            with open(report_csv, "w", newline='', encoding='utf-8') as csvfile:
                fieldnames = ["file_path", "status", "old_hash", "new_hash", "checked_at"]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for row in changes:
                    writer.writerow(row)
            print(f"[+] Report saved to {report_csv}")

def restore_files(backup_dir):
    if not backup_dir or not os.path.exists(backup_dir):
        print("[-] Backup directory not found.")
        return
    for root, _, files in os.walk(backup_dir):
        for file in files:
            backup_file_path = os.path.join(root, file)
            rel_path = os.path.relpath(backup_file_path, backup_dir)
            original_path = rel_path
            os.makedirs(os.path.dirname(original_path), exist_ok=True)
            shutil.copy2(backup_file_path, original_path)
            print(f"[RESTORED] {original_path}")

This function checks if the files saved in the database have changed or disappeared, optionally backs up any modified files, and can generate a CSV report listing all changes with timestamps.

If the database doesn’t exist, it prints an error and stops. Then it iterates through each file path stored in the database.
For each file, it sets a default status of "No Changes". If the file no longer exists on disk, the status is updated to "MISSING". If the file exists, the function calculates its current SHA-256 hash and compares it to the stored hash; if they differ, the status becomes "Changed".

if a backup directory is provided, any modified files are copied to the backup folder while preserving the folder structure. All changes including file path, status, old and new hashes, and the timestamp of the check are recorded in a list. 
Finally, if a CSV path is given, the function writes this list to a CSV file. This provides a complete, timestamped snapshot of which files are unchanged, missing, or modified, and optionally stores backups for recovery.

In [5]:
def restore_files(backup_dir):
    if not backup_dir or not os.path.exists(backup_dir):
        print("[-] Backup directory not found.")
        return
    for root, _, files in os.walk(backup_dir):
        for file in files:
            backup_file_path = os.path.join(root, file)
            rel_path = os.path.relpath(backup_file_path, backup_dir)
            original_path = rel_path
            os.makedirs(os.path.dirname(original_path), exist_ok=True)
            shutil.copy2(backup_file_path, original_path)
            print(f"[RESTORED] {original_path}")


This code first checks whether the given backup_dir exists; if not, it prints an error and exits. Then, it walks through all files inside the backup folder recursively. For each file, it calculates its relative path with respect to the backup directory to preserve the folder structure. It creates any necessary parent directories at the original location and copies the file from the backup to its original path. After each file is restored, it prints a message indicating the restoration. This ensures that all backed-up files can be returned to their original locations.

In [6]:
def run_file_integrity_tool():
    directory = input("Enter folder path")

    print("\n1. Build database")
    print("2. Check integrity")
    print("3. Restore files\n")

    choice = input("Select option (1/2/3): ")

    if choice == "1":
        if os.path.exists(directory):
            build_database(directory)
        else:
            print("[-] Invalid folder path")

    elif choice == "2":
        backup_dir = input("Enter backup folder path ").strip() or None
        report_csv = input("Enter CSV report path ").strip() or None
        check_integrity("integrity_db.json", backup_dir, report_csv)

    elif choice == "3":
        backup_dir = input("Enter backup folder path to restore from: ").strip()
        restore_files(backup_dir)

    else:
        print("[-] Invalid choice")


This block performs as the user interface.It prompts for a directory and a mode: "Build database", "Check integrity", or "restore files". 

In build mode, it scans all files, calculates their SHA-256 hashes, and saves them in a JSON database as a baseline. In check mode, it recalculates hashes, compares them to the database, marks missing or changed files, optionally backs up modified files, and can generate a CSV report with timestamps.

In restore mode, it copies files from a backup directory back to their original locations. The code also handles invalid directories or modes, providing informative messages, effectively coordinating scanning, verification, backup, reporting, and restoration in a single workflow.

In [7]:
input_folder = "/kaggle/input/information"

working_folder = "/kaggle/working/information"

shutil.copytree(input_folder, working_folder, dirs_exist_ok=True)

'/kaggle/working/information'

In [8]:
run_file_integrity_tool()

Enter folder path /kaggle/working/information



1. Build database
2. Check integrity
3. Restore files



Select option (1/2/3):  1


[+] Database created and saved to integrity_db.json


First , we created the dataset 

In [9]:
with open( "/kaggle/working/information/ABC.txt", "a", encoding="utf-8") as f:
    f.write("Some changes\n")  

Then , we made some changes in the file 

In [10]:
run_file_integrity_tool()

Enter folder path /kaggle/working/information



1. Build database
2. Check integrity
3. Restore files



Select option (1/2/3):  2
Enter backup folder path  /kaggle/working/backup
Enter CSV report path  /kaggle/working/report


[Changed] /kaggle/working/information/ABC.txt
[+] Report saved to /kaggle/working/report


As we can see , the cange was detected .
we stored define a path to sotore the backup and cv report .

In [11]:
run_file_integrity_tool()

Enter folder path /kaggle/working/information



1. Build database
2. Check integrity
3. Restore files



Select option (1/2/3):  3
Enter backup folder path to restore from:  /kaggle/working/backup


[RESTORED] information/ABC.txt


The original version at the time the database was created or at the moment the backup was made.

## Conclusion:


This tool reliably monitors file changes, detects modifications or missing files, creates backups, generates reports, and can restore files to their original state, helping maintain file integrity efficiently.